In [1]:
import requests
import xlwt
import xlrd
import csv
import pprint
import time
from IPython import embed
from tqdm import tqdm
wb = xlwt.Workbook()
ws = wb.add_sheet('广州',cell_overwrite_ok=True)

In [2]:
from geopy.distance import geodesic

In [3]:
import numpy as np

In [4]:
def poi_search(key,locations,types,radius):
    import json
    url_0 = 'https://restapi.amap.com/v3/place/around?'
    key = str(key)
    types = str(types)
    location = str(locations)
    radius = str(radius)
    extensions = 'all'

    url_lng_lat = url_0 + 'key=' + key + '&location=' + locations + '&types=' + types  + '&radius=' + radius + '&extensions=' + extensions
    data = []
    for p in range(1,50):
        url = url_lng_lat + '&page=' + str(p)
        html = requests.get(url)
        if html.status_code == 200:
            
            content = json.loads(html.text)
            dic = content["pois"]
            for i in dic:
                location = i["location"]
                name = i["name"]
                datas = [name,location]
                data.append(datas)

    return data

In [5]:
types = str(input("请输入你需要查询的公共服务设施类型：\n"))
weight = str(input("请输入权重：\n"))
#34 113.272673,23.096911 没爬完呢

请输入你需要查询的公共服务设施类型：
060100
请输入权重：
0.3


In [6]:
def poi_search_keywords(key,locations,keywords,radius):
    import json
    url_0 = 'https://restapi.amap.com/v3/place/around?'
    key = str(key)
    keywords = str(keywords)
    location = str(locations)
    radius = str(radius)
    extensions = 'all'

    url_lng_lat = url_0 + 'key=' + key + '&location=' + locations + '&keywords=' + keywords + '&radius=' + radius + '&extensions=' + extensions
    data = []
    for p in range(1,50):
        url = url_lng_lat + '&page=' + str(p)
        html = requests.get(url)
        if html.status_code == 200:
            
            content = json.loads(html.text)
            dic = content["pois"]
            for i in dic:
                location = i["location"]
                name = i["name"]
                datas = [name,location]
                data.append(datas)

    return data

In [7]:
keywords = str(input("请输入你需要查询的公共服务设施类型：\n"))
weight = str(input("请输入权重：\n"))

请输入你需要查询的公共服务设施类型：
儿童乐园
请输入权重：
0.05


In [6]:
def accessibility(key,origin,destination):
    import json
    url = "https://restapi.amap.com/v3/direction/walking?parameters"
    keyword = {"origin": origin,"destination": destination,"key": key,"output":"JSON"}
    access = []
    b = requests.get(url, params=keyword)
    if b.status_code == 200:
        html = b.text
        json = json.loads(html)
        route = json["route"]["paths"][0]
        distance = route["distance"]
        duration = route["duration"]
        access.append(distance)
        access.append(duration)
    return access


In [7]:
def transposexy(x):
    x_spit = x.split(",")
    x_spitnp = np.array(x_spit)
    #print(x_spitnp)
    x_spitnp[[0,1]] = x_spitnp[[1,0]]
    #print(x_spitnp)
    x_spitnp=','.join(x_spitnp)
    return x_spitnp

In [8]:
def csv_save(row):
    with open("data_address_gz_0.3.csv", 'a') as f:
        write = csv.writer(f)
        write.writerow(row)

def address_get():
    f = open(r'address_gz.txt', 'r')
    a = list(f)
    b = []
    for i in a:
        a = i.replace("  ", ",").replace("\n", "")
        b.append(a)
    return b

分类爬取

In [9]:
#if __name__ == '__main__':
print("【基础数据】")
key = str(input("请输入你的高德API密钥：\n"))
'''key = '7f4837aea08366cc4fd9a83c6b340bf7'

key = "a1cf852d9781d1e427a06ab57400ce13"

key = "153c2c2f24af71aeab07f04018732476"
ccc4642ebacd30e64fc4ef2ffa5e387c
'''
radius = int(input("请输入你需要查询的范围（单位：m）：\n"))
datas = []

address = address_get()
title = ["步行最短距离（m）","步行最短时间（s）","公共服务设施名称","ID",'距离分类','权重','纠正后权重']
datas.append(title)

【基础数据】
请输入你的高德API密钥：
ccc4642ebacd30e64fc4ef2ffa5e387c
请输入你需要查询的范围（单位：m）：
1100


In [11]:
print("\n【数据采集】")
print("数据采集进度：")
number = 0

for x in tqdm(address):
    #print(x)
    #print(type(x))
    number = number + 1
    a = poi_search(key, x,types,radius)
    #b = poi_search_keywords(key,x,keywords,radius)
    
    #print(a)
    for i in a:
        #a = accessibility(key, x, loca,headers)
        #print(a[0])
        #if int(a[0]) > 1305:
            #continue
        name = i[0]
        loca = i[1]
        #print(loca)
        #print(type(loca))
        #dist = geodesic((transposexy(x)), (transposexy(loca))).m
        a = accessibility(key, x, loca)
        print(a[0])
        if int(a[0]) > 1305:
            continue
        a.append(name)
        a.append(str(number))
        #a.append(str(dist))
        a0 = int(a[0])
        if a0 < 435:
            ind = 1
        elif a0 < 652.5 and a0 >= 435:
            ind = 0.9
        elif a0 < 870 and a0 >= 652.5:
            ind = 0.55
        elif a0 < 1087.5 and a0 >= 870:
            ind = 0.25
        elif a0 < 1305 and a0 >= 1087.5:
            ind = 0.08
            
        a.append(str(ind))
        a.append(str(weight))
        a.append(str(round((float(weight)*ind),4)))
        datas.append(a)
    '''  
    #print(b)
    for j in b:
        name = j[0]
        loca = j[1]
        a = accessibility(key, x, loca)
        print(a[0])
        if int(a[0]) > 1305:
            continue
        a.append(name)
        a.append(str(number))
        #a.append(str(dist))
        a0 = int(a[0])
        if a0 < 435:
            ind = 1
        elif a0 < 652.5 and a0 >= 435:
            ind = 0.9
        elif a0 < 870 and a0 >= 652.5:
            ind = 0.55
        elif a0 < 1087.5 and a0 >= 870:
            ind = 0.25
        elif a0 < 1305 and a0 >= 1087.5:
            ind = 0.08
            
        a.append(str(ind))
        a.append(str(weight))
        a.append(str(round((float(weight)*ind),4)))
        datas.append(a)
        '''

  0%|          | 0/171 [00:00<?, ?it/s]


【数据采集】
数据采集进度：
281
440
589
577
742
776
835
835
915
783
1007
1001
1057
1079
1033
1103
1185
1078
1298
1338
1275
1290
1418
1424
1443
1243
1387


  1%|          | 1/171 [00:16<46:02, 16.25s/it]

1367


  1%|          | 2/171 [00:26<35:11, 12.50s/it]

...

 98%|█████████▊| 168/171 [21:47<00:13,  4.66s/it]

161
1114
484
667
464
781
1106
909
828
1074
1232
1154
1161
1011
1011
1011
1407
1213
1272
1301
1217
1132
1081
1193
1239
1412


 99%|█████████▉| 169/171 [21:57<00:12,  6.33s/it]

1346
1697


100%|██████████| 171/171 [22:06<00:00,  7.76s/it]


In [12]:
for i in datas:
    csv_save(i)
print("\n-------------------数据采集完成-------------------")
pprint.pprint(datas)


-------------------数据采集完成-------------------
[['步行最短距离（m）', '步行最短时间（s）', '公共服务设施名称', 'ID', '距离分类', '权重', '纠正后权重'],
 ['281', '225', '粤海·得邻', '1', '1', '0.3', '0.3'],
 ['440', '352', '圣诚·商场', '1', '0.9', '0.3', '0.27'],
 ['589', '471', '广州亚太精品交易中心', '1', '0.9', '0.3', '0.27'],
 ['577', '462', '广州江湾阳光天地', '1', '0.9', '0.3', '0.27'],
 ['742', '594', '北京大厦饰博城', '1', '0.55', '0.3', '0.165'],
 ['776', '621', '信木岚(粤海仰忠汇店)', '1', '0.55', '0.3', '0.165'],
 ['835', '668', '天河城', '1', '0.55', '0.3', '0.165'],
 ['835', '668', '趣优游专柜', '1', '0.55', '0.3', '0.165'],
 ['915', '732', 'ABC-石烧安格斯牛扒(北京路店)', '1', '0.25', '0.3', '0.075'],
 ['783', '626', '骏田批发广场', '1', '0.55', '0.3', '0.165'],
 ['1007', '806', '荣戈百货', '1', '0.25', '0.3', '0.075'],
 ['1001', '801', '金佰利广场', '1', '0.25', '0.3', '0.075'],
 ['1057', '846', '仙妮蕾德(天河城百货店)', '1', '0.25', '0.3', '0.075'],
 ['1079', '863', '天河城百货(北京路店)', '1', '0.25', '0.3', '0.075'],
 ['1033', '826', '名盛广场', '1', '0.25', '0.3', '0.075'],
 ['1103', '882', '泰康仰忠精品批发城

1. 断线再运行可以继续写


开始打分